In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import quad

from mpl_toolkits.mplot3d import Axes3D

plt.switch_backend('qt5agg')

In [2]:
def integral(f, t0, tf):
    # f is a function
    return quad(f, t0, tf)[0]

# Initialize Parameters

In [3]:
# End time
#T = 65

# Income at time t, should be i(t)
def y(t):
    return 50000*np.exp(.03*(t))

# Non-risky Investment rate
r = .04

# Continuous process (think average return rate of risky investment)
mu = .09

# Standard deviation of risky investment
sigma = .18

# Utility discount rate
rho = .03

# Hazard Function (probability of death at this moment in time)
def lamb(t):
    return 1/200 + 9/8000*t

# Insurance premium-payout ratio
def eta(t):
    return 1/200 + 9/8000*t

# Relative risk aversion
gamma = -3

# Integral function doesn't play nice with sums of constants and functions
def sum_r(t):
    return r+eta(t)
def sum_rho(t):
    return lamb(t)+rho

def H(v):
    return sum_rho(v)/(1-gamma) - .5*gamma*((mu-r)/((1-gamma)*sigma))**2 - \
                                gamma/(1-gamma)*sum_r(v)

def K(s):
    return (lamb(s)**(1/(1-gamma)))/eta(s)**(gamma/(1-gamma)) + 1

def e(t,T):
    return np.exp(-integral(H,t,T)) + integral(lambda s: np.exp(-integral(H,t,s))*K(s),t,T)
    
def a(t,T):
    return np.exp(-rho*t)*e(t,T)**(1-gamma)

def b(t,T):
    return integral(lambda s: y(s)*np.exp(-integral(sum_r,t,s)),t,T)

def c(t,T):
    return (1/e(t,T))*(x+b(t,T))

def D(t,T):
    return (lamb(t)/eta(t))**(1/(1-gamma))*1/e(t,T)

def Z(t,T):
    return D(t,T)*(x+b(t,T))

def theta(t,T):
    return ((mu-r)/((1-gamma)*sigma**2))*(x+b(t,T))

def p(t,T):
    return eta(t)*((D(t,T)-1)*x+D(t,T)*b(t,T))

def dW(t):
    return np.random.normal(0,1)-np.random.normal(0,1)

# This is actually the wrong implementation

In [95]:
t0 = 0
T = 40
num_int = 400

# Since p(t) is the rate per year, 

# W0 = np.random.normal()
# W1 = np.random.normal()

timeline = np.linspace(t0,T,num_int+1)
dt = (T-t0)/num_int

future_wealth = b(0,T)
# future_wealth ~ 1200000
wealth = np.linspace(0, 3000000, 51) - 1100000

ps = []
cs = []
thetas = []
for w in wealth:
    p_star = []
    c_star = []
    theta_star = []
    x = w
    for t in timeline:
#         dW = W0 - W1
#         W0, W1 = W1, np.random.normal()
        z = p(t,T)
        chat = c(t,T)
        p_star.append(z)
        c_star.append(chat/(x+b(t,T)))
        theta_star.append(theta(t,T)/(x+b(t,T)))
        x += ((r*x - chat - z + y(t) + theta(t,T)*(mu-r))*dt)# + theta(t)*sigma*dW)
    ps.append(p_star)
    cs.append(c_star)
    thetas.append(theta_star)

# Hopefully this is the correct implementation

In [88]:
from tqdm import tqdm_notebook

In [89]:
# Need to redefine equations to avoid repeating calculations
def p(eta_, D_, b_, x):
    return eta_*((D_ - 1)*x + D_*b_)

def c(e_, x, b_):
    return 1/e_#*(x+b_)

In [90]:
# Starting Time
t0 = 0

# Finishing Time
T = 40

# Want dt=.01, so use (T-t0)/dt intervals
num_int = 400

timeline = np.linspace(t0,T,num_int+1)

# create vector of total wealth in the range [0, 3000000] as found in the paper
total_wealth = np.linspace(0,3000000,3001)

#create empty list to start collecting the p* values
ps = np.zeros((len(timeline), len(total_wealth)))
cs = np.zeros((len(timeline), len(total_wealth)))

for n, t in enumerate(timeline):
    # define the future wealth at time t
    b_ = b(t,T)
    
    # define the variables which don't change in t, this avoids unnecessary recalculations
    eta_ = eta(t)
    D_ = D(t,T)
    e_ = e(t,T)
    
    # Next we want to calculate p* everywhere
    # Need to define the current wealth
    x = total_wealth - b_
    
    p_star = p(eta_, D_, b_, x)
    ps[n] = p_star
    
    c_star = c(e_, x, b_)
    cs[n] = c_star

In [91]:
# Cross my fingers that it graphs properly
T_, W = np.meshgrid(timeline, total_wealth)

fig = plt.figure()
ax = fig.gca(projection='3d')
# ax.plot_surface(T, W, cs.T)
ax.plot_surface(T_, W, ps.T)
ax.invert_xaxis()
plt.show()

In [77]:
[D(t,T) for t in timeline]

[0.065381372869483659,
 0.065485899322825503,
 0.065590592962705391,
 0.065695455320472787,
 0.065800487946554242,
 0.065905692410703057,
 0.066011070302252617,
 0.066116623230373817,
 0.066222352824336056,
 0.066328260733772709,
 0.066434348628949952,
 0.066540618201040333,
 0.066647071162400151,
 0.066753709246851298,
 0.066860534209967362,
 0.066967547829364024,
 0.067074751904994234,
 0.067182148259447613,
 0.067289738738254737,
 0.067397525210196235,
 0.067505509567616279,
 0.067613693726741489,
 0.067722079628004486,
 0.067830669236372668,
 0.067939464541682204,
 0.068048467558977116,
 0.068157680328853817,
 0.068267104917811239,
 0.068376743418606237,
 0.068486597950615097,
 0.068596670660200157,
 0.068706963721082912,
 0.068817479334722806,
 0.068928219730702123,
 0.069039187167117169,
 0.069150383930975612,
 0.069261812338600459,
 0.069373474736040469,
 0.069485373499487149,
 0.06959751103569882,
 0.069709889782431325,
 0.069822512208875934,
 0.069935380816104439,
 0.070048498

# Figure 7. Optimal Life Insurance Rule

In [96]:
X, Y = np.meshgrid(wealth, timeline)

ps = np.array(ps)

fig = plt.figure()
ax = fig.gca(projection='3d')
ax.plot_surface(X, Y, ps.T)
plt.show()

# Figure 8. Optimal Consumption Proportion

In [ ]:
X, Y = np.meshgrid(wealth, timeline)

cs = np.array(cs)

fig = plt.figure()
ax = fig.gca(projection='3d')
ax.plot_surface(X, Y, cs.T)
plt.show()

# Figure 9. Optimal Portfolio Proportion

In [ ]:
X, Y = np.meshgrid(wealth, timeline)

thetas = np.array(thetas)

fig = plt.figure()
ax = fig.gca(projection='3d')
ax.plot_surface(X, Y, thetas.T)
plt.show()

# Figure 10. Critical level curve

In [ ]:
def f(t):
    return (lamb(t)/eta(t))**(1/(1-gamma))

def phi(t):
    return f(t)*b(t)/(a(t)**(1/(1-gamma))-f(t))

def phi2(t):
    return f(t)*b(t)/(e(t)-f(t))

crit = []
for t in timeline:
    crit.append(phi(t))

plt.plot(timeline, crit)
plt.show()

In [ ]:
b(0)

In [ ]:
a(0)

In [ ]:
e(0)